In [20]:
import pandas as pd
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK

In [21]:
df = pd.read_csv("../data/processed/customer_data_new_features.csv")

X = df.drop("churn", axis=1)
y = df["churn"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [22]:
def get_models():
    return {
        "LogisticRegression": LogisticRegression(max_iter=500),
        "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)
    }

In [23]:
def train_and_log(model, model_name, params=None):
    with mlflow.start_run(run_name=model_name):
        if params:
            model.set_params(**params)

        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy_score(y_test, preds)

      
        mlflow.log_metric("accuracy", acc)
        mlflow.log_param("model", model_name)
        if params:
            mlflow.log_params(params)

        mlflow.sklearn.log_model(model, model_name)

        print(f"{model_name} -> Accuracy: {acc:.4f}")
        return acc

In [24]:
def hyperopt_tune(model_name, max_evals=20):
    if model_name == "RandomForest":
        space = {
            "n_estimators": hp.choice("n_estimators", [50, 100, 200]),
            "max_depth": hp.choice("max_depth", [5, 10, 20, None]),
            "min_samples_split": hp.uniform("min_samples_split", 0.01, 0.2)
        }
        base_model = RandomForestClassifier(random_state=42)

    elif model_name == "XGBoost":
        space = {
            "n_estimators": hp.choice("n_estimators", [50, 100, 200]),
            "max_depth": hp.choice("max_depth", [3, 5, 7]),
            "learning_rate": hp.uniform("learning_rate", 0.01, 0.3)
        }
        base_model = XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42)

    else:
        print(f"No hyperopt space defined for {model_name}")
        return

    def objective(params):
        acc = train_and_log(base_model, f"{model_name}_tuned", params)
        return {"loss": -acc, "status": STATUS_OK}

    trials = Trials()
    best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    print(f"Best params for {model_name}: {best}")
    return best

In [28]:
mlflow.set_experiment("demo_churn")

models = get_models()
for name, model in models.items():
    train_and_log(model, name)

hyperopt_tune("RandomForest", max_evals=8)
hyperopt_tune("XGBoost", max_evals=8)

2025/09/28 18:26:29 INFO mlflow.tracking.fluent: Experiment with name 'demo_churn' does not exist. Creating a new experiment.
2025/09/28 18:26:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/28 18:26:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


LogisticRegression -> Accuracy: 0.8254


2025/09/28 18:26:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/28 18:26:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


RandomForest -> Accuracy: 0.8217


2025/09/28 18:26:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/28 18:26:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

2025/09/28 18:28:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/09/28 18:28:28 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



XGBoost_tuned -> Accuracy: 0.8300                                               
 88%|████████▊ | 7/8 [00:33<00:04,  4.89s/trial, best loss: -0.8373373373373373]

2025/09/28 18:28:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/09/28 18:28:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



XGBoost_tuned -> Accuracy: 0.8355                                               
100%|██████████| 8/8 [00:38<00:00,  4.82s/trial, best loss: -0.8373373373373373]
Best params for XGBoost: {'learning_rate': np.float64(0.1706869478158034), 'max_depth': np.int64(1), 'n_estimators': np.int64(0)}


{'learning_rate': np.float64(0.1706869478158034),
 'max_depth': np.int64(1),
 'n_estimators': np.int64(0)}

In [29]:
exp_name = "demo_churn"
experiment = mlflow.get_experiment_by_name(exp_name)
runs_df = mlflow.search_runs(experiment_ids=[experiment.experiment_id])

best_run = runs_df.loc[runs_df["metrics.accuracy"].idxmax()]
best_run_id = best_run.run_id

print("🔥 Best Run Details 🔥")
print(f"Run ID     : {best_run_id}")
print(f"Model Name : {best_run['params.model']}")
print(f"Accuracy   : {best_run['metrics.accuracy']:.4f}")

client = mlflow.tracking.MlflowClient()
params = client.get_run(best_run_id).data.params

print("\nHyperparameters:")
for k, v in params.items():
    if k != "model":  
        print(f"{k}: {v}")


🔥 Best Run Details 🔥
Run ID     : b92465da1fe942bcb1ee716e2267f081
Model Name : XGBoost_tuned
Accuracy   : 0.8373

Hyperparameters:
learning_rate: 0.1706869478158034
max_depth: 5
n_estimators: 50
